In [ ]:
import torch
print(torch.__version__) 
# 1.6.0이 안나온다면 아래 코드 실행 필요 

1.6.0+cu101


In [ ]:
!pip3 install torch==1.6.0+cu101 torchtext==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
# 1회만 실행할 것 
# 실행 후 런타임 재실행 필요 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s 
     |████████████████████████████████| 4.5 MB 13.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.


In [ ]:
import torch, torchtext
print(torch.__version__)     # 1.6.0+cu101
print(torchtext.__version__) # 0.7.0

1.6.0+cu101
0.7.0


In [ ]:
!cp drive/MyDrive/ai/GoogleNews-vectors-negative300.bin.gz .
!gunzip GoogleNews-vectors-negative300.bin.gz

In [ ]:
import os
import collections 
import nltk
nltk.download('punkt')

import numpy as np
import torch.nn as nn
import torch.optim as optim
import tqdm.notebook as tqdm

from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence

from torchtext.experimental.vocab import Vocab
from torchtext.experimental.datasets.text_classification import IMDB  #torch 안에 IMDB 데이터를 불러온다.

from gensim.models.keyedvectors import KeyedVectors

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 뉴럴 네트워크 하이퍼파라미터
num_classes = 2
num_epochs = 10
evaluate_per_steps = 100
learning_rate = 0.00005
dropout_rate = 0.4
batch_size = 50
weight_decay = 5e-4

# Text RNN 하이퍼파라미터
embedding_dim = 300
hidden_size = 200

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# RNN 모델

In [ ]:
class TextRnn(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, pretrained_embedding, hidden_size, dropout_rate):
        super().__init__()
        # embedding layer 선언 
        self.embedding = nn.Embedding(num_embeddings=vocab_size , embedding_dim=embedding_dim)
        # embedding layer에 pretrained_embedding 넣기 
        self.embedding.weight.detach().copy_(torch.tensor(pretrained_embedding))
        # rnn layer 선언 
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size)
        # dropout layer 선언 
        self.dropout =nn.Dropout(dropout_rate)
        # fc layer 선언 
        self.fc = nn.Linear(in_features=hidden_size, out_features=num_classes)

        #RNN의 기본 구조
    def forward(self, text, length):
        emb = self.embedding(text)
        out = pack_padded_sequence(emb, lengths=length, enforce_sorted=False)
        all_hidden_state, final_hidden_state = self.rnn(out) # final_hidden_state : [1, batch, hidden_dim]
        out = final_hidden_state[0]
        out = self.dropout(out)
        out = self.fc(out)
        return out 

In [ ]:
# batch:
# [
#   (tensor(0 or 1), tensor([3, 4, 13, 38, 20]),    5 단어
#   (tensor(0 or 1), tensor([21, 21, 3, 76]),       4 단어
#   (tensor(0 or 1), tensor([53, 48, 1]),           3 단어
#   (tensor(0 or 1), tensor([3, 4, 4, 1, 3, 20]),   6 단어
# ]

def collate(batch):
    labels, text = zip(*batch)

    lengths = []
    for t in text:
        lengths.append(len(t))

    text = pad_sequence(text) # 가장 긴 text를 기준으로 직사각형으로 만들어 줌
    lengths = torch.tensor(lengths) # List to tensor
    labels = torch.stack(labels) # stack : 1개의 차원을 추가해주면서 합치는 것 / 0 dim -> 1 dim(tensor)
    return labels, text, lengths

In [ ]:
def evaluate(model, data_loader):
    with torch.no_grad():
        model.eval()
        num_corrects = 0
        num_total = 0
        for label, text, length in data_loader:
            label = label.to(device)
            text = text.to(device)
            length = length.to(device)

            output = model(text, length)
            predictions = output.argmax(dim=-1)
            num_corrects += (predictions == label).sum().item()
            num_total += label.size(0)
            
        return num_corrects / num_total

# Embedding layer로 Google word2vec 사용

In [ ]:
def get_pre_trained_emb(vocab, embedding_dim, pre_trained_path):
    print('start to load google word2vec model')
    google_word2vec = KeyedVectors.load_word2vec_format(pre_trained_path, binary=True, limit=60000)
    print('Done!')

    matrix_len = len(vocab)
    weights_matrix = np.zeros((matrix_len, embedding_dim))
    words_found = 0

    for i, word in enumerate(vocab.itos): # itos : Index TO String
        try: 
            weights_matrix[i] = google_word2vec[word]
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.uniform(-0.2, 0.2, size=google_word2vec.vector_size)

    print(f'total vocab length : {matrix_len}, matched word count : {words_found}')
    return weights_matrix

In [ ]:
# 데이터 준비 
train_dataset, test_dataset = IMDB(tokenizer=None, data_select=("train", "test"))

vocab = train_dataset.get_vocab()
num_train = int(len(train_dataset)*0.9)
num_dev = len(train_dataset) - num_train

train_dataset, dev_dataset = random_split(train_dataset, (num_train, num_dev))
print(f'dataset size train / dev / test : {len(train_dataset)} / {len(dev_dataset)} / {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=batch_size    , num_workers=8, collate_fn=collate)
dev_loader   = DataLoader(dev_dataset,   batch_size=batch_size * 4, num_workers=8, collate_fn=collate)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size * 4, num_workers=8, collate_fn=collate)

25000lines [00:00, 27097.85lines/s]


dataset size train / dev / test : 22500 / 2500 / 25000


In [ ]:
# pre trained word embedding : Google word2vec 
vectors = get_pre_trained_emb(vocab=vocab, 
                              embedding_dim=embedding_dim, 
                              pre_trained_path='GoogleNews-vectors-negative300.bin')

start to load google word2vec model
Done!
total vocab length : 100684, matched word count : 23302


In [ ]:
# 모델 선언 
model = TextRnn(num_classes=num_classes,
                embedding_dim=embedding_dim,
                vocab_size=len(vocab),
                pretrained_embedding=vectors,
                hidden_size=hidden_size,
                dropout_rate=dropout_rate).to(device)

# 로스, 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 학습 진행
steps = 0
max_dev_accuracy = 0.0

for epoch in tqdm.trange(num_epochs):
    progress = tqdm.tqdm(train_loader)
    for label, text, length in progress:
        model.train()
        steps += 1
        label = label.to(device)
        text = text.to(device)
        length = length.to(device)

        output = model(text, length)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress.set_description(f'train loss: {loss.item():.4f}')

        if steps % evaluate_per_steps == 0:
            print('***** evaluating on the dev set *****')
            dev_accuracy = evaluate(model, dev_loader)
            print(f'dev accuracy: {dev_accuracy:.4f}')
            if dev_accuracy > max_dev_accuracy:
                max_dev_accuracy = dev_accuracy
                print('achieve dev-best accuracy. saving.')
                torch.save(model.state_dict(), 'best_weight.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.5424
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.5612
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.5720
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.5744
achieve dev-best accuracy. saving.


  0%|          | 0/450 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.7288
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8092
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8252
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.6844
***** evaluating on the dev set *****
dev accuracy: 0.8184


...

***** evaluating on the dev set *****
dev accuracy: 0.8740
***** evaluating on the dev set *****
dev accuracy: 0.8720
***** evaluating on the dev set *****
dev accuracy: 0.8768
***** evaluating on the dev set *****
dev accuracy: 0.8780
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.5736


  0%|          | 0/450 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.7656
***** evaluating on the dev set *****
dev accuracy: 0.8732
***** evaluating on the dev set *****
dev accuracy: 0.8816
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8764


  0%|          | 0/450 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.8724
***** evaluating on the dev set *****
dev accuracy: 0.8828
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8716
***** evaluating on the dev set *****
dev accuracy: 0.5100
***** evaluating on the dev set *****
dev accuracy: 0.6792


In [ ]:
# 평가
print('***** evaluating dev-best on the test set *****')
model.load_state_dict(torch.load('best_weight.pt'))
test_accuracy = evaluate(model, test_loader)
print(f'test accuracy: {test_accuracy}')

***** evaluating dev-best on the test set *****
test accuracy: 0.86964
